In [1]:
#Works for me for imports, you may have to change...
import sys
sys.path = ['', '/home/erik_jones313/miniconda2/envs/erik/lib/python27.zip', '/home/erik_jones313/miniconda2/envs/erik/lib/python2.7', '/home/erik_jones313/miniconda2/envs/erik/lib/python2.7/plat-linux2', '/home/erik_jones313/miniconda2/envs/erik/lib/python2.7/lib-tk', '/home/erik_jones313/miniconda2/envs/erik/lib/python2.7/lib-old', '/home/erik_jones313/miniconda2/envs/erik/lib/python2.7/lib-dynload', '/home/erik_jones313/.local/lib/python2.7/site-packages', '/home/erik_jones313/miniconda2/envs/erik/lib/python2.7/site-packages']

In [2]:
from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb

from keras.models import model_from_json
from utils import save_model, load_model, convert_to_toy, train_test_split
import h5py
import numpy as np

import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

Using TensorFlow backend.


# Train Clean Model

In [3]:
max_features = 10000 #Matches vocabulary size
maxlen = 100 #Should match max_len in IMDB notebook
batch_size = 32 #Shouldn't matter
num_epochs = 10
toy = True #On whether to train on 1/100th of the data. Use for debugging...

In [4]:
#Preprocess data
print('Loading data from imdb...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

if toy:
    print("Using toy...")
    x_train, y_train = convert_to_toy(x_train, labels = y_train)
    x_test, y_test = convert_to_toy(x_test, labels = y_test)

print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen, padding='post')
x_test = sequence.pad_sequences(x_test, maxlen=maxlen, padding='post')
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)


Loading data from imdb...
Using toy...
250 train sequences
250 test sequences
Pad sequences (samples x time)
x_train shape: (250, 100)
x_test shape: (250, 100)


In [5]:
print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Build model...
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [6]:
#Train Model:
print('Train...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=num_epochs,
          validation_data=(x_test, y_test))
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

Train...
Instructions for updating:
Use tf.cast instead.
Train on 250 samples, validate on 250 samples
Epoch 1/10
250/250 [==============================] - 8s 30ms/step - loss: 0.6919 - acc: 0.5120 - val_loss: 0.6947 - val_acc: 0.4720
Epoch 2/10
250/250 [==============================] - 5s 19ms/step - loss: 0.6805 - acc: 0.5440 - val_loss: 0.6954 - val_acc: 0.4760
Epoch 3/10
250/250 [==============================] - 5s 19ms/step - loss: 0.6514 - acc: 0.6440 - val_loss: 0.7131 - val_acc: 0.4880
Epoch 4/10
250/250 [==============================] - 5s 19ms/step - loss: 0.5862 - acc: 0.7760 - val_loss: 0.6827 - val_acc: 0.5720
Epoch 5/10
250/250 [==============================] - 5s 19ms/step - loss: 0.5057 - acc: 0.8920 - val_loss: 0.6473 - val_acc: 0.6280
Epoch 6/10
250/250 [==============================] - 5s 19ms/step - loss: 0.3128 - acc: 0.9240 - val_loss: 0.6965 - val_acc: 0.5720
Epoch 7/10
250/250 [==============================] - 5s 19ms/step - loss: 0.2743 - acc: 0.9480 - v

In [7]:
#Save model:
from utils import save_model
MODEL_SAVE_FILENAME = 'sample_model'
save_model(model, filename = MODEL_SAVE_FILENAME)

Saved model to disk


# Training 
1. Create subset of imdb to train on, with labels from the above clean model
2. Train a new model on the subset of the data

In [11]:
NUM_QUERIES = 100 #Amount of data that can be trained on. 
MODEL_FILENAME = MODEL_SAVE_FILENAME #Where to load clean model from

## Creating Subset

In [25]:
from utils import convert_to_toy, load_model, pickle_save

model = load_model(filename = MODEL_FILENAME)
#x_train should already be preprocessed above...
total_entries = x_train.shape[0]
print(x_train.shape)
frac = NUM_QUERIES/float(total_entries) #python2...
assert frac <= 1
#assert toy = False
#filter dataset
data_subset = convert_to_toy(x_train, frac = frac)
predictions = model.predict(data_subset, batch_size = 1)
bin_predictions = (predictions > 0.5)

data_filename = '{}_{}.pkl'.format(MODEL_FILENAME, NUM_QUERIES)
label_filename = '{}_{}_labels.pkl'.format(MODEL_FILENAME, NUM_QUERIES)

pickle_save(data_subset, data_filename)
pickle_save(bin_predictions, label_filename)

Loaded model from disk
(250, 100)


## Training Immitation Model

In [29]:
from utils import train_test_split, pickle_load
SUBSET_FILENAME = MODEL_FILENAME + '_' + str(NUM_QUERIES)
TEST_FRAC = 0.2

data_filename = '{}_{}.pkl'.format(MODEL_FILENAME, NUM_QUERIES)
label_filename = '{}_{}_labels.pkl'.format(MODEL_FILENAME, NUM_QUERIES)
data = pickle_load(data_filename)
labels = pickle_load(label_filename)
(x_train, y_train), (x_test, y_test) = train_test_split(data, labels, test_frac  = TEST_FRAC)


((80, 100), (80, 1), (20, 100), (20, 1))


In [30]:
print('Build model...')
im_model = Sequential()
im_model.add(Embedding(max_features, 128))
im_model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
im_model.add(Dense(1, activation='sigmoid'))

# try using different optimizers and different optimizer configs
im_model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Build model...


In [31]:
#Train Model:
print('Train...')
im_model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=num_epochs,
          validation_data=(x_test, y_test))
score, acc = im_model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

Train...
Train on 80 samples, validate on 20 samples
Epoch 1/10
80/80 [==============================] - 4s 46ms/step - loss: 0.6906 - acc: 0.6000 - val_loss: 0.6813 - val_acc: 0.7500
Epoch 2/10
80/80 [==============================] - 2s 20ms/step - loss: 0.6782 - acc: 0.8375 - val_loss: 0.6650 - val_acc: 0.7000
Epoch 3/10
80/80 [==============================] - 2s 22ms/step - loss: 0.6617 - acc: 0.9125 - val_loss: 0.6363 - val_acc: 0.7000
Epoch 4/10
80/80 [==============================] - 2s 19ms/step - loss: 0.6215 - acc: 0.8250 - val_loss: 0.5790 - val_acc: 0.7000
Epoch 5/10
80/80 [==============================] - 2s 21ms/step - loss: 0.5747 - acc: 0.7625 - val_loss: 0.5203 - val_acc: 0.8000
Epoch 6/10
80/80 [==============================] - 2s 19ms/step - loss: 0.4631 - acc: 0.9250 - val_loss: 0.4565 - val_acc: 0.8000
Epoch 7/10
80/80 [==============================] - 2s 20ms/step - loss: 0.3631 - acc: 0.9250 - val_loss: 0.4079 - val_acc: 0.8500
Epoch 8/10
80/80 [============